In [1]:
import torch
import os,sys; sys.path.insert(0, os.path.abspath('.'))

sys.path.append('../../')
from hex.pytorch.board_graph import Board, PositionalEncoder
from hex.pytorch.graph_net import GraphNet
from utils import dotdict

ModuleNotFoundError: No module named 'hex.pytorch'

In [12]:
import os

if not os.path.isdir('./dtest/x'):
    if not os.path.isdir('./dtest'):
        os.mkdir('./dtest')
    os.mkdir('./dtest/x')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './dtest'

In [5]:
a = torch.tensor([0,5,1])
torch.stack((a, a*2), dim=0).min(dim=0)

torch.return_types.min(
values=tensor([0, 5, 1]),
indices=tensor([1, 0, 0]))

In [40]:
[0] * 3

[0, 0, 0]

In [ ]:
b = Board(torch.zeros(4,4).long())
b.np_pieces[1,0] = -1
b.np_pieces[1,1] = -1
b.np_pieces[2,1] = -1
b.np_pieces[2,2] = -1
b.np_pieces[0,2] = 1
b.np_pieces[0,3] = 1
b.np_pieces[3,2] = 1
b.np_pieces[2,0] = 1
print(b.display_string, "\n\n", b.np_pieces)

In [ ]:
args = dotdict({
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 10,
    'batch_size': 64,
    'cuda': torch.cuda.is_available(),
    'num_channels': 32,
    'res_blocks' : 5,
    'board_size' : 7,
    #'in_channels' : 3,                   # 0/1/2 - black/white/empty
    'expand_base' : 2,
    'attn_heads' : 1,
    'pos_encoding_sz' : 28,
    'readout_attn_heads' : 4
})

model = GraphNet(args)

In [ ]:
p, v = model(b.np_pieces.unsqueeze(dim=0))

In [ ]:
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

penc = PositionalEncoder(d_model=28, max_seq_len=500, base_wave_length=5)
#x = torch.randperm(20)
x = torch.arange(500)
x = penc(x)

ax = sns.heatmap(x[:20].numpy(), cmap=sns.color_palette("Blues"))

In [ ]:
torch.rand(4, 4).softmax(dim=1)

In [ ]:
import numpy as np
import torch

In [ ]:
a = np.arange(4)
for i, p in enumerate(a):
    print(i, p)

In [ ]:
a = torch.arange(4)
str(a.numpy())

In [ ]:
d = {1: 2, 2:3}
len(d)

In [ ]:
 %matplotlib
 import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ln, = ax.plot(range(5))


In [ ]:
ln.set_color('orange')

In [ ]:
plt.ioff()

In [ ]:
plt.ion()

In [ ]:
ln.set_color('red')

In [ ]:
import logging

log = logging.getLogger()

In [ ]:
log.warn('hi')

In [ ]:
log.level = logging.info()
log

In [ ]:
from absl import app, flags, logging

In [ ]:
d = {'board_type': 'vortex', 'player2': 'nnet', 'num_games': 2, 'cpu1_checkpoint': 'temp/gat/strong_5x5_b.pth.tar', 'cpu2_checkpoint': 'temp/gat/strong_5x5_b.pth.tar', 'p1_MCTS_sims': 2, 'p2_MCTS_sims': 2, 'game_board_size': 5, 'p1_nnet': 'base_gat', 'p2_nnet': 'base_gat', 'git_record': 'commit c77717f47f31eb09a63d8bea3147aa3c01b07847\nAuthor: Phil Hawkins <phil.a.hawkins@gmail.com>\nDate:   Thu Jul 30 20:04:34 2020 +1000\n\n    refactor with vortex game\n', 'p1_wins': 1, 'p2_wins': 1}

In [ ]:
d

In [ ]:
import ast

s = "{'board_type': 'vortex', 'player2': 'nnet', 'num_games': 2, 'cpu1_checkpoint': 'temp/gat/strong_5x5_b.pth.tar', 'cpu2_checkpoint': 'temp/gat/strong_5x5_b.pth.tar', 'p1_MCTS_sims': 2, 'p2_MCTS_sims': 2, 'game_board_size': 5, 'p1_nnet': 'base_gat', 'p2_nnet': 'base_gat', 'git_record': 'commit c77717f47f31eb09a63d8bea3147aa3c01b07847\nAuthor: Phil Hawkins <phil.a.hawkins@gmail.com>\nDate:   Thu Jul 30 20:04:34 2020 +1000\n\n    refactor with vortex game\n', 'p1_wins': 1, 'p2_wins': 1}"
s = s.replace('\n', '\\n')
d = ast.literal_eval(s)
d

In [ ]:
import networkx as nx
import numpy as np
import scipy as sp
import torch
from pyvis import network as net


In [ ]:
bsize = 4
n = bsize**2
G = nx.grid_2d_graph(bsize, bsize)

layout = {}
for i in range(bsize):
    for j in range (bsize):
        if i < (bsize-1) and j > 0:
            G.add_edge((i, j), (i+1, j-1))
        layout[(i, j)] = (j + (i/2), bsize-1-i)
nx.draw_networkx(G, layout)

In [ ]:
A = nx.to_scipy_sparse_matrix(G)
edge_index = torch.tensor(A.nonzero())
A = torch.sparse_coo_tensor(indices=edge_index, values=torch.ones_like(edge_index[0]), size=(n, n))
Ad = A.to_dense()
edge_index, n

In [ ]:
# connectivity distance 1
# tree depth 2
A1c = torch.einsum('ij,jk->ikj',Ad,Ad)
A1 = ((A1c.sum(dim=2) > 1).long().fill_diagonal_(0) - Ad).relu()
A1c = A1c * A1.unsqueeze(dim=2)

Ad, A1, A1c[0, 5]

In [ ]:
A1c[0, 5], A1c[5, 10]

In [ ]:
def and_rule(v1, v2):
    vi = (v1*v2).max() + 1
    vm = (v1.sum() - vi) * (v2.sum() - vi)
    return min(max(vm, 0), 1)

In [ ]:
and_rule(A1c[0, 5], A1c[5, 10])

In [ ]:
and_rule(A1c[1, 6], A1c[6, 8])

In [ ]:
from multiprocessing import Pool



In [2]:
def f(x):
    return x*x

if __name__ == '__main__':
    with Pool() as p:
        print(p.map(f, [1, 2, 3]))